In [42]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')




# A/B тест баннера в блоке "Скидки недели" на главной странице 

Необходимо провести A/B тестирование новой реализации блока "Скидки недели" на главной странице.

Ссылка на макет: https://www.figma.com/file/nPUhnmVBRBfVY4JQWNGvcj/Главная-страница?node-id=1397%3A26441

КОНТРОЛЬНАЯ группа: Показываем версию главной страницы с двухэтажным блоком скидки недели, но без красного баннера.
ТЕСТОВАЯ группа: показываем версию главной страницы с двухэтажным блоком скидки недели и с красным баннером.

КУКИ:
exp_id=ab_0034
exp_var=1 // Контрольная группа
exp_var=2 // Тестовая группа

В текущей версии сайта в блоке "Скидки недели" кнопка для перехода на полный листинг товаров со скидками не слишком заметка пользователем, что приводит к тому, что пользователи не так часто как хотелось бы просматривают остальные товары со скидками, которые в теории могли бы заинтересовать их.
В новой версии главной страницы вместо кнопки используется привлекающий к себе внимание красный баннер с призывом перейти в листинг товара со скидками.

Гипотеза: Новая реализация кнопки для перехода в виде баннера увеличит CR по переходу в листинг со скидками, увеличит взаимодействие с листингом товаров со скидками, а также благоприятно скажется на прибыли компании.


Метрики: 
- CR в уникальные клики
- Распределение кликов по товарам в блоке и кнопке для перехода в листинг в %

Вспомогательные метрики: 
- CR в добавления в корзину


In [43]:
import pandas as pd
from google.oauth2 import service_account

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import ztest
from statsmodels.stats import weightstats
from datetime import datetime
import statsmodels.stats.weightstats as ws
import math 
import warnings

warnings.filterwarnings('ignore')


In [44]:
# Прописываем адрес к файлу с данными по сервисному аккаунту и получаем credentials для доступа к данным
credentials = service_account.Credentials.from_service_account_file(
    '/Users/lxndrarura/Python Santehnika/key/spry-compound-139714-3c5694cec4ca.json')

project_id = 'spry-compound-139714' 



In [45]:
# Формируем запрос и получаем количество вопросов с тегом "pandas", сгруппированные по дате создания
query = '''
with
ab_1 as(
select 
    ab.date,
    t.clientId,
    ab.sessionId,
    ab.experimentId,
    ab.experimentVariant
from `spry-compound-139714.Presets.presets_ab` as ab
join `spry-compound-139714.Presets.presets_traffic` t on t.sessionId = ab.sessionId 
where ab.experimentId = 'ab_0034'
and ab.experimentVariant in ('1', '2')
and ab.date between '2022-01-01' and '2022-01-30'
and t.date between '2022-01-01' and '2022-01-30'
and ab.sessionId in (select d.sessionId from `spry-compound-139714.Presets.presets_devices` as d where d.deviceCategory = 'desktop' and d.date between '2022-01-01' and '2022-01-30')
and ab.sessionId not in (select f.sessionId from `spry-compound-139714.Presets.presets_filters` as f)),

-- # дубликаты сессий
duplicates as(
select 
    sessionid,
    count(sessionId) as s_count
from ab_1
group by 1
having s_count >1),

# пользоваьели у которых были дублирующиеся сессии
users_for_remove as (
select
    distinct(ab.clientId)
from ab_1 as ab 
join duplicates d on d.sessionId = ab.sessionId

),

# сессии без юзеров с дублирующимися сессиями 
ab as (
select ab_1.* from ab_1
where ab_1.clientId not in (select clientId from users_for_remove)
)

select 
    distinct(i.sessionId),
    i.date,
    ab.experimentVariant
from
    `spry-compound-139714.Presets.presets_product_impressions` as i 
join 
    ab 
    on ab.sessionId = i.sessionId
where 
    i.impressionList = 'Week Discounts'
    and i.date between '2022-01-01' and '2022-01-30'


'''


# Выполняем запрос с помощью функции ((https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_gbq.html read_gbq)) в pandas, записывая результат в dataframe

views = pd.read_gbq(query, project_id=project_id, credentials=credentials)

# display(views.head(10))
# print(views.info())




In [46]:
# Формируем запрос и получаем количество вопросов с тегом "pandas", сгруппированные по дате создания
query = '''
with
ab_1 as(
select 
    ab.date,
    t.clientId,
    ab.sessionId,
    ab.experimentId,
    ab.experimentVariant
from `spry-compound-139714.Presets.presets_ab` as ab
join `spry-compound-139714.Presets.presets_traffic` t on t.sessionId = ab.sessionId 
where ab.experimentId = 'ab_0034'
and ab.experimentVariant in ('1', '2')
and ab.date between '2022-01-01' and '2022-01-30'
and t.date between '2022-01-01' and '2022-01-30'
and ab.sessionId in (select d.sessionId from `spry-compound-139714.Presets.presets_devices` as d where d.deviceCategory = 'desktop' and d.date between '2022-01-01' and '2022-01-30')
and ab.sessionId not in (select f.sessionId from `spry-compound-139714.Presets.presets_filters` as f)),

-- # дубликаты сессий
duplicates as(
select 
    sessionid,
    count(sessionId) as s_count
from ab_1
group by 1
having s_count >1),

# пользоваьели у которых были дублирующиеся сессии
users_for_remove as (
select
    distinct(ab.clientId)
from ab_1 as ab 
join duplicates d on d.sessionId = ab.sessionId

),

# сессии без юзеров с дублирующимися сессиями 
ab as (
select ab_1.* from ab_1
where ab_1.clientId not in (select clientId from users_for_remove)
)

select 
    p.sessionId,
    p.hitId,
    case 
        when p.productName = 'Больше товаров' then 'Больше товаров'
        when p.productName != 'Больше товаров' then p.position
    end position,
    p.date,
    ab.experimentVariant
from 
    `spry-compound-139714.Presets.presets_product_clicks` as p 
join 
    ab 
    on ab.sessionId = p.sessionId
where 
    p.date between '2022-01-01' and '2022-01-30'
    and p.productListName = 'Week Discounts'

'''


# Выполняем запрос с помощью функции ((https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_gbq.html read_gbq)) в pandas, записывая результат в dataframe

clicks = pd.read_gbq(query, project_id=project_id, credentials=credentials)

# display(clicks.head(10))
# print(clicks.info())





In [47]:
# Формируем запрос и получаем количество вопросов с тегом "pandas", сгруппированные по дате создания
query = '''
with
ab_1 as(
select 
    ab.date,
    t.clientId,
    ab.sessionId,
    ab.experimentId,
    ab.experimentVariant
from `spry-compound-139714.Presets.presets_ab` as ab
join `spry-compound-139714.Presets.presets_traffic` t on t.sessionId = ab.sessionId 
where ab.experimentId = 'ab_0034'
and ab.experimentVariant in ('1', '2')
and ab.date between '2022-01-01' and '2022-01-30'
and t.date between '2022-01-01' and '2022-01-30'
and ab.sessionId in (select d.sessionId from `spry-compound-139714.Presets.presets_devices` as d where d.deviceCategory = 'desktop' and d.date between '2022-01-01' and '2022-01-30')
and ab.sessionId not in (select f.sessionId from `spry-compound-139714.Presets.presets_filters` as f)),

-- # дубликаты сессий
duplicates as(
select 
    sessionid,
    count(sessionId) as s_count
from ab_1
group by 1
having s_count >1),

# пользоваьели у которых были дублирующиеся сессии
users_for_remove as (
select
    distinct(ab.clientId)
from ab_1 as ab 
join duplicates d on d.sessionId = ab.sessionId

),

# сессии без юзеров с дублирующимися сессиями 
ab as (
select ab_1.* from ab_1
where ab_1.clientId not in (select clientId from users_for_remove)
),

clicks as (
select 
    p.sessionId,
    p.hitId,
    case 
        when p.productName = 'Больше товаров' then 'Больше товаров'
        when p.productName != 'Больше товаров' then p.position
    end position,
    p.date,
    ab.experimentVariant
from 
    `spry-compound-139714.Presets.presets_product_clicks` as p 
join 
    ab 
    on ab.sessionId = p.sessionId
where 
    p.date between '2022-01-01' and '2022-01-30'
    and p.productListName = 'Week Discounts'
), 

t1 as (
select 
    h.sessionId,
    h.hitId,
    h.eventAction,
    h.date,
    h.timestamp,
    case 
        when h.eventAction = 'main_card_weekDiscounts_click' then 1 
        else 0 
    end mark,
    ab.experimentVariant
from 
    `spry-compound-139714.Presets.presets_hits_new` as h
join
    ab 
    on ab.sessionId = h.sessionId
where 
    h.date between '2022-01-01' and '2022-01-30'
    and (h.eventAction = 'add' or h.eventAction = 'main_card_weekDiscounts_click')
order by 
    h.sessionId, h.timestamp
),

t2 as (
select 
    h.sessionId,
    h.hitId,
    h.eventAction,
    h.date,
    h.timestamp,
    h.mark,
    h.experimentVariant,
    case 
        when h.eventAction = 'add' and (max(mark) OVER(partition by h.sessionId ORDER BY h.timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW))=1 
        then 'yes'
        else 'no'
    end mark1
from 
    t1 as h 
where 
    h.sessionId in (select sessionId from clicks)
order by 
    h.sessionId, h.timestamp
)

-- добавления в корзину 

select 
    h.sessionId,
    h.hitId,
    h.date,
    h.experimentVariant,
from t2 as h 
where 
    mark1 = 'yes'

'''


# Выполняем запрос с помощью функции ((https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_gbq.html read_gbq)) в pandas, записывая результат в dataframe

carts = pd.read_gbq(query, project_id=project_id, credentials=credentials)

# display(carts.head(10))
# print(carts.info())





In [48]:
res = views.groupby('experimentVariant')['sessionId'].nunique().reset_index()
res['%GT'] = (res['sessionId']/res['sessionId'].sum())*100



In [49]:
clicks.groupby('experimentVariant')['sessionId'].nunique().reset_index().rename(columns={'sessionId':'Уникальные клики по блоку'})
clicks[clicks['position']=='Больше товаров'].groupby('experimentVariant')['sessionId'].nunique().reset_index().rename(columns={'sessionId':'Уникальные клики по больше товаров'})

res = res.merge(clicks.groupby('experimentVariant')['sessionId'].nunique().reset_index().rename(columns={'sessionId':'Все уникальные клики'}), on = 'experimentVariant', how='left')
res = res.merge(clicks[clicks['position']=='Больше товаров'].groupby('experimentVariant')['sessionId'].nunique().reset_index().rename(columns={'sessionId':'Уникальные клики по больше товаров'}), on = 'experimentVariant', how='left')



In [50]:
res['Общий CR в уник. клики, %'] = (res['Все уникальные клики']/res['sessionId'])*100
res['CR в уникальные клики по больше товаров, %'] = (res['Уникальные клики по больше товаров']/res['sessionId'])*100




In [51]:
carts.groupby('experimentVariant')['sessionId'].nunique().reset_index().rename(columns={'sessionId':'Добавления в корзину после взаимодействия с блоком'})

res = res.merge(carts.groupby('experimentVariant')['sessionId'].nunique().reset_index().rename(columns={'sessionId':'Добавления в корзину после взаимодействия с блоком'}), on ='experimentVariant' , how='left')




In [52]:
res['CR в корзину из просмотра блока, %'] = (res['Добавления в корзину после взаимодействия с блоком'] / res['sessionId'])*100




In [53]:
res['CR в корзину из клика по блоку, %'] = (res['Добавления в корзину после взаимодействия с блоком']/res['Все уникальные клики'])*100




In [54]:
# res.round(2)
# res

# r['diff,%'].apply(lambda x: round(x, 2))


for i in res.iloc[:, 1:]:
    res[i] = res[i].apply(lambda x: round(x, 2))






In [55]:
resT = res.T
resT.columns = resT.iloc[0]
resT = resT.iloc[1:]

resT = resT.rename(columns = {"1":'control', "2":'test'})
resT['%diff'] = (100/resT['control'])*resT['test']-100
resT.columns.name=None





## Сравнение метрик

In [56]:
import plotly.graph_objects as go
metricks = resT
metricks = metricks.round(decimals=2)

fig = go.Figure(data=[go.Table(
 # columnorder = [1,4],
  columnwidth = [200,100,100,100],
  header = dict(
    values = ['','control', 'test', '%diff'],
    line_color='darkslategray',
    fill_color='rgb(103,112,244)',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=40
  ),
  cells=dict(
    values=[metricks.index, metricks['control'], metricks['test'], metricks['%diff']],
    line_color='darkslategray',
    fill=dict(color=[
        ['rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', '#a1bcf7', '#a1bcf7', 'rgb(229, 238, 255)','#a1bcf7','#a1bcf7'],
        ['white','white','white', 'white','#d9d9d9','#d9d9d9', 'white', '#d9d9d9','#d9d9d9'],
        ['white','white','white', 'white','#d9d9d9','#d9d9d9', 'white', '#d9d9d9','#d9d9d9'],
        ['white','white','white', 'white','#f57c73','#d9d9d9', 'white', '#d9d9d9','#fff2cc']]),
    # font=dict(color=['rgb(25, 25, 112)', ['rgb(25, 25, 112)', 'rgb(25, 25, 112)', 'rgb(25, 25, 112)', 'rgb(128, 128, 128)', 'rgb(25, 25, 112)'], ['rgb(25, 25, 112)', 'rgb(25, 25, 112)', 'rgb(25, 25, 112)', 'rgb(128, 128, 128)', 'rgb(25, 25, 112)'], ['rgb(25, 25, 112)', 'rgb(25, 25, 112)', 'rgb(25, 25, 112)', 'rgb(128, 128, 128)','rgb(25, 25, 112)']]),
    align=['left', 'center'],
    font_size=12,
    height=30)
    )
])
fig.show()






In [57]:
clicks.groupby(['experimentVariant', 'position'])['sessionId'].nunique().reset_index().sort_values(by=['experimentVariant', 'position'], ascending=True)

# clicks[clicks['position']!='Больше товаров']['position'] = clicks[clicks['position']!='Больше товаров']['position'].apply(lambda x: int(x))

cl = clicks[clicks['position']!='Больше товаров']
cl['position'] = cl['position'].apply(lambda x: int(x))

cl = cl.groupby(['experimentVariant', 'position'])['sessionId'].nunique().reset_index().sort_values(by=['experimentVariant', 'position'], ascending=True)
cl1 = clicks[clicks['position']=='Больше товаров']
cl1['position'] = 19
cl1 = cl1.groupby(['experimentVariant', 'position'])['sessionId'].nunique().reset_index().sort_values(by=['experimentVariant', 'position'], ascending=True)

cl = pd.concat([cl, cl1], ignore_index=True)

cl = cl.rename(columns = {'sessionId': 'Уникальные клики'})
cl = cl.sort_values(by=['experimentVariant', 'position'], ascending=True)



In [58]:
vs = views.groupby('experimentVariant')['sessionId'].nunique().reset_index()
cl['views'] = cl['experimentVariant'].apply(lambda x: int(vs[vs['experimentVariant']=='1']['sessionId']) if x=='1' else int(vs[vs['experimentVariant']=='2']['sessionId']))





In [59]:
cl['CR,%'] = (cl['Уникальные клики']/cl['views'])*100



## Сравнение CR в клики по позициям в блоке скидки недели 

Позиция 19 - это кнопка "Больше товаров"


In [60]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = go.Figure()
# fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=cl[cl['experimentVariant']=='1']['position'], y=cl[cl['experimentVariant']=='1']['CR,%'], name="CR,% control", text=cl[cl['experimentVariant']=='1']['CR,%'], textposition='outside')
)

fig.add_trace(
    go.Bar(x=cl[cl['experimentVariant']=='2']['position'], y=cl[cl['experimentVariant']=='2']['CR,%'], name="CR,% test", text=cl[cl['experimentVariant']=='2']['CR,%'], textposition='outside')
)

# fig.add_trace(
#     go.Scatter(x=dist[dist['experimentVariant']==1]['position'], y=dist[dist['experimentVariant']==1]['CR_click%'], name="CR в клики control"),
#     secondary_y=True,
# )

# fig.add_trace(
#     go.Scatter(x=dist[dist['experimentVariant']==2]['position'], y=dist[dist['experimentVariant']==2]['CR_click%'], name="CR в клики test"),
#     secondary_y=True,
# )


# Add figure title
fig.update_layout(
    title_text="CR по позициям",
    xaxis = dict(title= 'Позиции',ticklen= 5,zeroline= False, tickmode='linear'), 
 #   yaxis = dict(title= 'Клики',ticklen= 5,zeroline= False),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Set y-axes titles
# fig.update_yaxes(title_text="CR в клики, %", secondary_y=True)
# fig.update_yaxes(title_text="Клики", secondary_y=False)


fig.show()




